In [0]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [0]:
#column_names = ["age","workclass","education","education_num","marital_status","occupation","relationship","race","gender","capital_gain",
#              "capital_loss","hours_per_week","native_country","income_bracket"]
url ="https://raw.githubusercontent.com/aj1Sharma/logistic_income/master/census_data.csv"
data = pd.read_csv(url)

In [66]:
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age               32561 non-null int64
workclass         32561 non-null object
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
gender            32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income_bracket    32561 non-null object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


## Converting output to Numeric Form

In [68]:
data['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [0]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [0]:
data['income_bracket'] = data['income_bracket'].apply(label_fix)

### Perform a Train Test Split on the Data

In [0]:
x_data = data.drop('income_bracket',axis=1)
y_labels = data['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

### Working on tf.esitmator


In [72]:
data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

#### Create numeric columns for the categorical values.

In [0]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [0]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [0]:
feat_cols = [gender, occupation, marital_status, relationship, education, workclass, native_country,
            age, education_num, capital_gain, capital_loss, hours_per_week]

### Create Input Function

In [0]:
input_func=tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size = 100, num_epochs = None, shuffle = True)

#### Model using Linear Classifier..Can also be done by DNN Classifier


In [77]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

W0811 16:48:08.681869 140116940220288 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpe8b1r8vx


** Train your model on the data, for at least 5000 steps. **

In [79]:
model.train(input_fn = input_func, steps = 5000)

### Evaluation

In [0]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x = X_test,batch_size = len(X_test), shuffle = False)

In [0]:
predictions = list(model.predict(input_fn = pred_fn))

In [82]:
predictions[0]

{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.2772823], dtype=float32),
 'logits': array([-0.9579828], dtype=float32),
 'probabilities': array([0.72271776, 0.27728227], dtype=float32)}

In [0]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [84]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

## Classification Report

In [85]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90      7436
           1       0.72      0.62      0.66      2333

    accuracy                           0.85      9769
   macro avg       0.80      0.77      0.78      9769
weighted avg       0.84      0.85      0.85      9769

